In [ ]:
import setup  # notebooks/setup.py
setup.init()

In [ ]:
from mypermit import permit_client

permit = permit_client

In [ ]:
from django.contrib.auth import get_user_model
from documents.models import Document


User = get_user_model()

admin_users = User.objects.filter(is_superuser=True)
documents = Document.objects.all()

In [ ]:
from permit import PermitApiError

async def get_or_create_resource_instance(django_instance, resource_key="document"):
    _key = f"{resource_key}:{django_instance.id}"
    try:
        _resource = await permit.api.resource_instances.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            payload = {
                "key": f"{django_instance.id}",
                "resource": resource_key,
                "tenant": "default",
                
            }
            _resource = await permit.api.resource_instances.create(payload)
        else:
            raise e
    return _resource

In [ ]:
async def assign_user_ownership(django_user, django_instance, resource_key="document"):
    _key = f"{resource_key}:{django_instance.id}"
    return await permit.api.users.assign_role({
        "user": f"{django_user.id}",
        "role": "admin",
        "resource_instance": _key,
        "tenant": "default",
    })

In [ ]:
for obj in documents:
    response = await get_or_create_resource_instance(obj, resource_key="document")
    role = await assign_user_ownership(obj.owner, obj, resource_key="document")
    print(response)

In [ ]:
for admin_u in admin_users:
    for obj in documents:
        response = await get_or_create_resource_instance(obj, resource_key="document")
        role = await assign_user_ownership(admin_u, obj, resource_key="document")
        print(response)

In [ ]:
random_user = User.objects.all().order_by("?")[0]
random_doc = Document.objects.all().order_by("?")[0]

await permit.check(f"{random_user.id}", "update", f"document:{random_doc.id}")